In [ ]:
import pandas as pd

# Investment and Operation Planning Module

This example considers a network of 5 nodes, with a load connected at each node. One representative day with with operational hours is used to describe the potential operational scenarios. Three types of investment candidate generators i.e, battery units, solar units and diesel generation units. We showcase how the Investment and Operation Planning Modules of PyEPLAN can be used to derive optimal units to be installed in the network.

## Importing the invesment and operation planning module from PyEPLAN 

In [ ]:
from pyeplan import inosys

## Defining the input data

The next step is to define the directory to the folder where the input data is located and define the reference node. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).

In [ ]:
inosys = inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

Some of the data inputs include:

### Defining the total load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [ ]:
pd.read_csv("1_bus/prep_dist.csv")

,0
0,70


### Defining the input investment candidates

#### Battery Units

In [ ]:
pd.read_csv("1_bus/cwin_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,186,0,0,0,50,0,0
1,0,186,0,0,0,50,0,0


#### Solar PV Units

In [ ]:
pd.read_csv("1_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,109,0,0,0,50,0,0
1,0,109,0,0,0,50,0,0


#### Diesel/Fossil Units

In [ ]:
pd.read_csv("1_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12,0.4,0,0,50,0,0,NaN
1,0,12,0.4,0,0,50,0,0,NaN


## Solving the optimization problem

PyEPLAN can be used to solve the problem investment and operation planning problems simultaneously. In case no investment candidates are availble, a sole operation planning problem can be run by setting input 'onlyopr = True'. Available solvers inclde both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. If discrete capacities of investment units are available the input 'invest = True' this sets the investement-related decision variables to a binary nature where the unit capacity = nominal. Otherwise,  'invest = False' sets the investement-related decision variables to a continuous nature where the unit capacity <= nominal.

In [ ]:
inosys.solve(solver = 'glpk', onlyopr = False, invest = True, )

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\BLESSI~1\AppData\Local\Temp\tmpzw_rtsm4.glpk.raw --wglp
 C:\Users\BLESSI~1\AppData\Local\Temp\tmpwvpxx5sa.glpk.glp --cpxlp C:\Users\BLESSI~1\AppData\Local\Temp\tmp99e21yty.pyomo.lp
Reading problem data from 'C:\Users\BLESSI~1\AppData\Local\Temp\tmp99e21yty.pyomo.lp'...
C:\Users\BLESSI~1\AppData\Local\Temp\tmp99e21yty.pyomo.lp:252: warning: lower bound of variable 'xg(0)' redefined
C:\Users\BLESSI~1\AppData\Local\Temp\tmp99e21yty.pyomo.lp:252: warning: upper bound of variable 'xg(0)' redefined
42 rows, 31 columns, 85 non-zeros
6 integer variables, all of which are binary
258 lines were read
Writing problem data to 'C:\Users\BLESSI~1\AppData\Local\Temp\tmpwvpxx5sa.glpk.glp'...
221 lines were written
GLPK Integer Optimizer 5.0
42 rows, 31 columns, 85 non-zeros
6 integer variables, all of which are binary
Preprocessing...
11 rows, 16 columns, 34 non-zeros
6 integer variables, all of which are binar

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total Investment and Operational Costs

In [ ]:
pd.read_csv("1_bus/results/obj.csv")

,total costs,10430.0
0,total investment costs,6050.0
1,total operation costs,4380.0


### Number and capacity of wind units installed

In [ ]:
pd.read_csv("1_bus/results/xw.csv")

,0,1
0,0.0,0.0


The capacity and location of wind units installed is:

In [ ]:
cwin = pd.read_csv("1_bus/cwin_dist.csv")
iwin = pd.read_csv("1_bus/results/xw.csv")
bus = cwin.loc[:,'bus']
(((cwin.loc[:,'pmax']*round(iwin.loc[0:,].T,2))[0]).to_frame().set_index(bus)).rename(columns={0: 'Installed Capacity (kW)'})

,Installed Capacity (kW)
bus,
0,0.0
0,0.0


### Number and capacity of solar units installed

In [ ]:
pd.read_csv("1_bus/results/xs.csv")

,0,1
0,1.0,0.0


The capacity and location of solar units installed is:

In [ ]:
csol = pd.read_csv("1_bus/csol_dist.csv")
isol = pd.read_csv("1_bus/results/xs.csv")
bus = csol.loc[:,'bus']
(((csol.loc[:,'pmax']*round(isol.loc[0:,].T,2))[0]).to_frame().set_index(bus)).rename(columns={0: 'Installed Capacity (kW)'})

,Installed Capacity (kW)
bus,
0,50.0
0,0.0


### Number and capacity of diesel units installed

In [ ]:
pd.read_csv("1_bus/results/xg.csv")

,0,1
0,1.0,0.0


The capacity and location of diesel units installed is:

In [ ]:
cgen = pd.read_csv("1_bus/cgen_dist.csv")
igen = pd.read_csv("1_bus/results/xg.csv")
bus = cgen.loc[:,'bus']
(((cgen.loc[:,'pmax']*round(igen.loc[0:,].T,2))[0]).to_frame().set_index(bus)).rename(columns={0: 'Installed Capacity (kW)'})

,Installed Capacity (kW)
bus,
0,50.0
0,0.0


### Amount of load curtailed

The level of load demand that has been curtailed at each node is:

In [ ]:
pd.read_csv("1_bus/results/pds.csv")

,0
0,0.0
